## <p style="text-align: center;">Numerical Algorithms - Homework Assignment 3</p>
**<p style="text-align: center;">VU Numerical Algorithms, summer semester 2018. Due to 17.06.2018. </p>**

### Programming Exercises

#### Effects of Preconditioners on Conjugate Gradient (12 points)

Implement the conjugate gradient method *CG* and experimentally investigate the effect of different preconditioners on its convergence. Please compare standard *CG* and preconditioned *CG* (*PCG*) for three given symmetric positive definite test problems in terms of the number of iterations and in terms of the runtime (including the time for
computing and applying the preconditioner!) until convergence. Show the convergence histories (norm of relative residual vs. iteration number) graphically.

* Implement standard CG efficiently (do not use the CG implementation available in Octave!). In particular, store the sparse matrix in a sparse matrix format
* Use the following preconditioners:
Diagonal preconditioner
  * Block diagonal preconditioner
  * Incomplete Cholesky factorization with no fill-in
  * Incomplete Cholesky factorization with threshold dropping: experiment with different thresholds and discuss the effect of the choice of threshold.
* Test matrices: Please use (at least) the following three test matrices from the
"Matrix Market" (http://math.nist.gov/MatrixMarket/) for your experiments:
  * http://math.nist.gov/MatrixMarket/data/Harwell-Boeing/lanpro/nos5.html
  * http://math.nist.gov/MatrixMarket/data/Harwell-Boeing/lanpro/nos6.html
  * http://math.nist.gov/MatrixMarket/data/misc/cylshell/s3rmt3m3.html

In [38]:
%plot -f svg

% Import routines needed to execute assignment.
source("source/assignment3.m")
% Import routines for reading matrix market (.mtx) format.
% Source: https://math.nist.gov/MatrixMarket/mmio/matlab/mmiomatlab.html.
source("source/mminfo.m")
source("source/mmread.m")

% CG with preconditioning.
function [numberOfIterations, runTime, relResHistory] = applyPCCG(A, convergenceThreshold, precondMethod)
    tic;
    
    numberOfIterations = 0;
    relResHistory = 0;
    
    runTime = toc;
end

% Generate matrix M for preconditioning of matrix A used in applyCG(...).
% Note: dropThreshold is ignored for all methods except cholesky_drop.
function M = generatePreconditioning(method, dropThreshold = 0)
    M = 0;
end

% Execute code for programming assignment.
function execute()
    % Iterate over test matrices.
    filenames = {"nos5.mtx"; "nos6.mtx"; "s3rmt3m3.mtx"};
    for i = 1:length(filenames)
        % Load file.
        [A, rows, cols, entries, rep, field, symm] = mmread(["data/" filenames{i}]);

        % Prepare data structure for evaluation metrics.
        res.st = struct();
        res.pc = struct();
        
        % ----------------------------------------------------
        % 1. Apply standard CG.
        % ----------------------------------------------------
        [
            res.st.numberOfIterations, ...
            res.st.runTime, ...
            res.st.relativeResidualHistory
        ] = applyCG(A, 10^-3);
        
        
        % ----------------------------------------------------
        % 2. Apply CG with preconditioner.
        % ----------------------------------------------------
        pcNames = {"diag"; "block"; "cholesky_nofi"; "cholesky_drop"};
        for i = 1:length(pcNames)
            % Generate matrix for preconditioning.
            M = generatePreconditioning(pcNames{i}, 1);
            % Note: Best drop threshold found in 3. is used here.
            [
                res.pc.(pcNames{i}).numberOfIterations, ...
                res.pc.(pcNames{i}).runTime, ...
                res.pc.(pcNames{i}).relativeResidualHistory
            ] = applyPCCG(A, 10^-3, 42);
        end
        res
        
        % ----------------------------------------------------
        % 3. Investigate effect of threshold for PC with 
        % incomplete Cholesky factorization with threshold 
        % dropping.
        % ----------------------------------------------------
    end
end

execute()

res =

  scalar structure containing the fields:

    st =

      scalar structure containing the fields:

        numberOfIterations =  25
        runTime =  0.0029540
        relativeResidualHistory =

         Columns 1 through 5:

           5.1123e-01   1.3282e-01   5.4330e-02   2.8981e-02   1.8046e-02

         Columns 6 through 10:

           1.5165e-02   1.4195e-02   1.1537e-02   9.9514e-03   6.6836e-03

         Columns 11 through 15:

           4.7755e-03   3.6828e-03   3.1388e-03   2.5123e-03   2.1511e-03

         Columns 16 through 20:

           2.7462e-03   2.7306e-03   1.9559e-03   1.4221e-03   1.2655e-03

         Columns 21 through 25:

           1.2515e-03   1.4508e-03   1.7792e-03   1.7674e-03   1.3821e-03

         Column 26:

           9.6671e-04


    pc =

      scalar structure containing the fields:

        diag =

          scalar structure containing the fields:

            numberOfIterations: 1x1 scalar
            runTime: 1x1 scalar
            rel


         Columns 221 through 225:

           2.7490e-03   2.8116e-03   2.8328e-03   2.7521e-03   2.6053e-03

         Columns 226 through 230:

           2.6068e-03   2.4795e-03   2.3860e-03   2.3946e-03   2.4199e-03

         Columns 231 through 235:

           2.4515e-03   2.4558e-03   2.5682e-03   2.5175e-03   2.3724e-03

         Columns 236 through 240:

           2.1793e-03   2.0980e-03   1.9941e-03   1.7846e-03   1.6815e-03

         Columns 241 through 245:

           1.6216e-03   1.5821e-03   1.4581e-03   1.4150e-03   1.3935e-03

         Columns 246 through 250:

           1.3920e-03   1.3812e-03   1.3548e-03   1.3519e-03   1.3381e-03

         Columns 251 through 255:

           1.3623e-03   1.3670e-03   1.4762e-03   1.5642e-03   1.4489e-03

         Columns 256 through 260:

           1.3857e-03   1.4230e-03   1.4802e-03   1.5387e-03   1.6451e-03

         Columns 261 through 265:

           1.7126e-03   1.7698e-03   1.6790e-03   1.6396e-03   1.5647e-03

         